In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, LongType, IntegerType, DoubleType

In [2]:
spark = SparkSession.builder.appName("Automotive Scenario").getOrCreate()

In [3]:
order_schema = StructType([
    StructField('OrderID',StringType(),True),
    StructField('CompanyID',StringType(),True),
    StructField('PartsRequired',StringType(),True),
    StructField('OrderQuantity',LongType(),True),
    StructField('CostOfPart',DoubleType(),True),
    StructField('OrderDate',StringType(),True),
    StructField('ExpectedDeliveryDate',StringType(),True),
    StructField('DeliveryStatus',StringType(),True)
])

In [4]:
company_schema = StructType([
    StructField('CompanyID',StringType(),True),
    StructField('CompanyName',StringType(),True),
    StructField('CompanyLocation',StringType(),True),
    StructField('CompanyContact',LongType(),True),
    StructField('EstablishedYear',IntegerType(),True)
])

In [5]:
order_book_df = spark.read.csv('./OrderBook.csv',schema=order_schema)

In [6]:
company_details_df = spark.read.csv('./CompanyDetails.csv',schema=company_schema)

In [7]:
order_book_df.show()

+-------+---------+--------------------+-------------+----------+---------+--------------------+--------------+
|OrderID|CompanyID|       PartsRequired|OrderQuantity|CostOfPart|OrderDate|ExpectedDeliveryDate|DeliveryStatus|
+-------+---------+--------------------+-------------+----------+---------+--------------------+--------------+
|OD10001|    C1108|        Spring Forks|        21103|9487867.48| 3-Nov-19|           24-Dec-19|       Pending|
|OD10002|    C1035|              Fender|        26272|4899369.32|18-Dec-19|           14-Mar-20|       Pending|
|OD10003|    C1174|           Generator|        39789|4008848.49| 7-Dec-19|           31-Jan-20|     Delivered|
|OD10004|    C1121|              Brakes|        22670|7932390.15|21-Nov-19|            5-Mar-20|       Pending|
|OD10005|    C1093|       Exhaust Pipes|        36773|6101829.61| 8-Dec-19|            9-Mar-20|       Pending|
|OD10006|    C1166|       Exhaust Pipes|        46571|8704566.05|19-Dec-19|           10-Jan-20|       P

In [8]:
company_details_df.show()

+---------+----------------+---------------+--------------+---------------+
|CompanyID|     CompanyName|CompanyLocation|CompanyContact|EstablishedYear|
+---------+----------------+---------------+--------------+---------------+
|    C1001|    Steel Wheels|   Pennsylvania|    7172809057|           2007|
|    C1002|   MotoLab Bikes|          Texas|    6829309477|           2011|
|    C1003|        Lee Auto|     California|    6193601290|           1994|
|    C1004|     Zoom Dynamo|          Texas|    8327309957|           1993|
|    C1005|Quakesons Motors|          Texas|    8175902345|           2002|
|    C1006|    Steel Wheels|   Pennsylvania|    5706889534|           2009|
|    C1007|     Raido Bikes|         Hawaii|    8086946394|           2001|
|    C1008|    Steel Wheels|      Tennessee|    6155558344|           2012|
|    C1009| Ilmio Jane Moto|   Pennsylvania|    6108768782|           2007|
|    C1010|      Reo Bailio|         Kansas|    7857454105|           2012|
|    C1011| 

In [9]:
company_details_df.createOrReplaceTempView('Company_table')

In [10]:
old_comp = spark.sql('select * from Company_table where EstablishedYear <= 2008')

In [11]:
from pyspark.sql.functions import col, expr

In [12]:
old_comp.show()

+---------+----------------+---------------+--------------+---------------+
|CompanyID|     CompanyName|CompanyLocation|CompanyContact|EstablishedYear|
+---------+----------------+---------------+--------------+---------------+
|    C1001|    Steel Wheels|   Pennsylvania|    7172809057|           2007|
|    C1003|        Lee Auto|     California|    6193601290|           1994|
|    C1004|     Zoom Dynamo|          Texas|    8327309957|           1993|
|    C1005|Quakesons Motors|          Texas|    8175902345|           2002|
|    C1007|     Raido Bikes|         Hawaii|    8086946394|           2001|
|    C1009| Ilmio Jane Moto|   Pennsylvania|    6108768782|           2007|
|    C1011|    Steel Wheels|        Arizona|    6026446720|           1992|
|    C1012|Wingmo Standards|     Washington|    5091629062|           2006|
|    C1013|    Craiko Roads|           Iowa|    3191050692|           1990|
|    C1014|      Reo Bailio|     California|    6195580058|           1995|
|    C1015| 

In [13]:
combined_df = order_book_df.join(company_details_df,on='CompanyID')

In [14]:
combined_df.show()

+---------+-------+--------------------+-------------+----------+---------+--------------------+--------------+----------------+---------------+--------------+---------------+
|CompanyID|OrderID|       PartsRequired|OrderQuantity|CostOfPart|OrderDate|ExpectedDeliveryDate|DeliveryStatus|     CompanyName|CompanyLocation|CompanyContact|EstablishedYear|
+---------+-------+--------------------+-------------+----------+---------+--------------------+--------------+----------------+---------------+--------------+---------------+
|    C1108|OD10001|        Spring Forks|        21103|9487867.48| 3-Nov-19|           24-Dec-19|       Pending|     Raido Bikes|        Georgia|    9125860714|           1995|
|    C1035|OD10002|              Fender|        26272|4899369.32|18-Dec-19|           14-Mar-20|       Pending| Ilmio Jane Moto| North Carolina|    9105420265|           2000|
|    C1174|OD10003|           Generator|        39789|4008848.49| 7-Dec-19|           31-Jan-20|     Delivered|   MotoLa

In [15]:
combined_df.createOrReplaceTempView('combined_table')

In [16]:
brakes_only = spark.sql("select CompanyName from combined_table where PartsRequired == 'Brakes'")

In [17]:
brakes_only.show()

+------------------+
|       CompanyName|
+------------------+
|          Lee Auto|
|         Rigmarole|
|       Wicker Auto|
|        Kioto Moto|
|  Quakesons Motors|
|     MotoLab Bikes|
|           Neopins|
|      Laser Wheels|
|         Rigmarole|
|Maino World Motors|
|Maino World Motors|
|      Craiko Roads|
|      Steel Wheels|
|       Raido Bikes|
|           Neopins|
|       Zoom Dynamo|
|      Laser Wheels|
|        Kioto Moto|
|   Ilmio Jane Moto|
|        Reo Bailio|
+------------------+
only showing top 20 rows



In [18]:
exhaust = spark.sql("select count(PartsRequired) from combined_table where PartsRequired == 'Exhaust Pipes' AND CompanyLocation == 'New York' ")

In [19]:
exhaust.show()

+--------------------+
|count(PartsRequired)|
+--------------------+
|                   2|
+--------------------+



In [20]:
delivered = spark.sql('select count(DeliveryStatus) from combined_table where DeliveryStatus == "Delivered"')

In [21]:
delivered.show()

+---------------------+
|count(DeliveryStatus)|
+---------------------+
|                  255|
+---------------------+



In [22]:
north_cal = spark.sql("select sum(OrderQuantity), sum(CostOfPart) from combined_table where CompanyLocation == 'North Carolina'")

In [23]:
north_cal.show()

+------------------+---------------+
|sum(OrderQuantity)|sum(CostOfPart)|
+------------------+---------------+
|            974157| 1.4936626449E8|
+------------------+---------------+



In [24]:
exhaust_dates = spark.sql("select OrderDate AS StartDate, ExpectedDeliveryDate AS EndDate, CompanyName from combined_table where PartsRequired == 'Exhaust Pipes'")

In [25]:
exhaust_dates.show()

+---------+---------+------------------+
|StartDate|  EndDate|       CompanyName|
+---------+---------+------------------+
| 8-Dec-19| 9-Mar-20|  Quakesons Motors|
|19-Dec-19|10-Jan-20|         Rigmarole|
|18-Oct-19|13-Mar-20|Maino World Motors|
|26-Nov-19|22-Dec-19|       Zoom Dynamo|
|20-Oct-19| 2-Jan-20|          Lee Auto|
|12-Nov-19| 8-Jan-20|Maino World Motors|
|30-Oct-19| 7-Feb-20|       Raido Bikes|
|11-Oct-19|11-Mar-20|Nando Kewillswitch|
|27-Oct-19|28-Jan-20|   Ilmio Jane Moto|
|25-Oct-19|28-Feb-20|Maino World Motors|
|18-Nov-19| 8-Mar-20|Maino World Motors|
|14-Oct-19| 6-Mar-20|  Quakesons Motors|
| 5-Nov-19|14-Jan-20|         Rigmarole|
| 8-Dec-19|12-Jan-20|      Craiko Roads|
| 9-Nov-19|10-Mar-20|      Damson Bikes|
|27-Oct-19|25-Dec-19|      Steel Wheels|
|19-Dec-19|17-Feb-20|  Quakesons Motors|
|24-Oct-19|22-Mar-20|        Reo Bailio|
| 8-Dec-19|25-Jan-20|  Quakesons Motors|
| 6-Dec-19|14-Jan-20|        Reo Bailio|
+---------+---------+------------------+
only showing top